In [1]:
import numpy as np  
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats
import math

In [2]:
class NaiveBayesClassifier:
    def __init__(self):
        self.classes = None
        self.means = None
        self.variances = None
        self.priors = None
        self.M_s=0
        self.Perror=0
        self.M = None

    def fit(self, X, y):
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_features = X.shape[1]
        self.means = np.zeros((n_classes, n_features))
        self.variances = np.zeros((n_classes, n_features))   
        self.priors = np.zeros(n_classes)
    
        # 計算先驗機率
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.priors[i] = X_c.shape[0] / X.shape[0]
            if X_c.shape[0] == 0:
                self.means[i] = np.zeros(n_features)
                self.variances[i] = np.zeros(n_features)
            else:
                self.means[i] = np.mean(X_c, axis=0)
                self.variances[i] = np.var(X_c, axis=0) + 1e-9

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1) + 1

    def score(self,X,labels_X):
        predict = self.predict(X)
        result = [labels_X[i] == predict[i] for i in range(len(X))]
        return np.sum(result)/len(X)

    def predict_proba(self, X):
        likelihood = np.zeros((X.shape[0], len(self.classes)))
        for i, c in enumerate(self.classes):
            class_likelihood = stats.norm.pdf(X, loc=self.means[i], scale=np.sqrt(self.variances[i]))
            class_likelihood = np.prod(class_likelihood, axis=1)
            likelihood[:, i] = class_likelihood * self.priors[i]
        evidence = np.sum(likelihood * self.priors, axis=1) + 1e-10
        return likelihood / evidence.reshape(-1, 1)

    def fit2(self, X, y):
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_features = X.shape[1]
        self.means = np.zeros((n_classes, n_features))
        self.variances = np.zeros((n_classes, n_features, n_features))   
        self.priors = np.zeros(n_classes)

        # 計算先驗機率
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.priors[i] = X_c.shape[0] / X.shape[0]
            if X_c.shape[0] == 0:
                self.means[i] = np.zeros(n_features)
                self.variances[i] = np.zeros((n_features, n_features))
            else:
                self.means[i] = np.mean(X_c, axis=0)
                self.variances[i] = np.cov(X_c, rowvar=False) + 1e-6*np.eye(n_features)
    

    def Bayes_error(self):
        # Initialize M matrix
        n_classes = len(self.classes)
    
        if len(self.variances.shape) == 1:
            self.variances = self.variances.reshape(-1, 1)

        s_values = np.linspace(0.1, 0.9, 20)

        # Find the optimal s value
        opt_s = 0
        min_error = float('inf')

        for s in s_values:
            Perrors = []
            for i in range(n_classes):
                for j in range(i+1, n_classes):
                    variance1 = self.variances[i]
                    variance2 = self.variances[j]
                    det1 = np.linalg.det(variance1)
                    det2 = np.linalg.det(variance2)
                    M1 = self.means[i]
                    M2 = self.means[j]
                    diff = M2 - M1
                    term1 = np.dot(np.dot(diff.T, np.linalg.inv(s * variance1 + (1 - s) * variance2)), diff)
                    term2 = s * (1 - s) * np.log(det2 / det1)
                    Perror = 0.5 * term1 + 0.25 * term2
                    Perrors.append(Perror)

            # Output the results
            print(f"For s={s}:")
            for i in range(n_classes):
                for j in range(i+1, n_classes):
                    print(f"Perror({i+1},{j+1}) = {Perrors[i*(n_classes-1)+(j-i-1)]}")

            # Update the optimal s value and corresponding error rate
            max_idx = np.argmax(Perrors)
            if Perrors[max_idx] < min_error:
                min_error = Perrors[max_idx]
                opt_s = s

        print(f"Optimal s value: {opt_s}")
        print(f"Minimum error rate: {min_error}")


In [3]:
# 指定特徵名稱
feature_names = ['label',
                 'Alcohol', 
                 'Malic acid',
                 'Ash',
                 'Alcalinity of ash' ,
                 'Magnesium',
                 'Total phenols',
                 'Flavanoids',
                 'Nonflavanoid phenols',
                 'Proanthocyanins',
                 'Color intensity',
                 'Hue',
                 'OD280/OD315 of diluted wines',
                 'Proline' ]
data=pd.read_csv("wine.data", names=feature_names)
data=data.sample(frac=1)
half = len(data) // 2
print(data.head()) # show first 5 items.
print(data.info())

     label  Alcohol  Malic acid   Ash  Alcalinity of ash  Magnesium   
146      3    13.88        5.04  2.23               20.0         80  \
112      2    11.76        2.68  2.92               20.0        103   
131      3    12.88        2.99  2.40               20.0        104   
63       2    12.37        1.13  2.16               19.0         87   
50       1    13.05        1.73  2.04               12.4         92   

     Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins   
146           0.98        0.34                  0.40             0.68  \
112           1.75        2.03                  0.60             1.05   
131           1.30        1.22                  0.24             0.83   
63            3.50        3.10                  0.19             1.87   
50            2.72        3.27                  0.17             2.91   

     Color intensity   Hue  OD280/OD315 of diluted wines  Proline  
146             4.90  0.58                          1.33      415 

In [4]:
train=data.iloc[:half]
test=data.iloc[half:]
print(type(train),train.shape)
print(type(test),test.shape)
print(train)

<class 'pandas.core.frame.DataFrame'> (89, 14)
<class 'pandas.core.frame.DataFrame'> (89, 14)
     label  Alcohol  Malic acid   Ash  Alcalinity of ash  Magnesium   
146      3    13.88        5.04  2.23               20.0         80  \
112      2    11.76        2.68  2.92               20.0        103   
131      3    12.88        2.99  2.40               20.0        104   
63       2    12.37        1.13  2.16               19.0         87   
50       1    13.05        1.73  2.04               12.4         92   
..     ...      ...         ...   ...                ...        ...   
97       2    12.29        1.41  1.98               16.0         85   
172      3    14.16        2.51  2.48               20.0         91   
102      2    12.34        2.45  2.46               21.0         98   
90       2    12.08        1.83  2.32               18.5         81   
40       1    13.56        1.71  2.31               16.2        117   

     Total phenols  Flavanoids  Nonflavanoid phenols 

In [5]:
# 抽出標籤那一行
labels = train.iloc[:,0]
labels_test= test.iloc[:,0]
print(labels.head())
# 初始化標準化器
scaler = StandardScaler()
# 對特徵值進行標準化
print(train)
print(train.shape)
labels = train.iloc[:,0].values
labels_test = test.iloc[:,0].values
train = train.drop("label",axis=1)
test  = test.drop("label",axis=1)
train_features = scaler.fit_transform(train.iloc[:,:])
test_features  = scaler.transform(test.iloc[:, :])
print(labels)
print(labels.shape)
print(test_features)
print(test_features.shape)

146    3
112    2
131    3
63     2
50     1
Name: label, dtype: int64
     label  Alcohol  Malic acid   Ash  Alcalinity of ash  Magnesium   
146      3    13.88        5.04  2.23               20.0         80  \
112      2    11.76        2.68  2.92               20.0        103   
131      3    12.88        2.99  2.40               20.0        104   
63       2    12.37        1.13  2.16               19.0         87   
50       1    13.05        1.73  2.04               12.4         92   
..     ...      ...         ...   ...                ...        ...   
97       2    12.29        1.41  1.98               16.0         85   
172      3    14.16        2.51  2.48               20.0         91   
102      2    12.34        2.45  2.46               21.0         98   
90       2    12.08        1.83  2.32               18.5         81   
40       1    13.56        1.71  2.31               16.2        117   

     Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins   
146

Hw1結果

In [6]:
nb = NaiveBayesClassifier()
nb.fit(train_features, labels)
x1=nb.predict(test_features)
print(x1)
print(labels_test)
print(nb.score(test_features,labels_test))


[1 3 3 2 3 1 3 2 2 1 2 2 3 1 2 2 1 1 2 3 2 2 2 1 3 2 3 3 1 3 2 3 1 1 2 3 1
 1 1 1 2 3 3 1 1 2 2 2 2 2 1 3 3 1 3 1 1 2 3 1 1 2 3 2 1 2 2 3 1 2 1 2 2 1
 2 2 2 2 2 2 1 2 3 3 1 2 2 2 2]
[1 3 3 2 3 1 3 2 2 1 2 2 3 1 2 2 1 1 2 3 2 2 2 1 3 2 3 3 1 3 2 3 1 1 2 3 1
 1 1 1 2 3 3 1 1 2 2 1 2 2 1 3 3 1 3 1 1 2 3 1 1 2 3 2 1 2 2 3 1 2 1 2 2 1
 2 2 2 2 2 2 1 2 3 2 1 2 2 2 2]
0.9775280898876404


Hw2結果

In [7]:
nb.fit2(test_features, labels)
nb.Bayes_error()


For s=0.1:
Perror(1,2) = 0.929082425327936
Perror(1,3) = 1.0095277185754232
Perror(2,3) = 0.6353748053691525
For s=0.14210526315789473:
Perror(1,2) = 0.8522754193550013
Perror(1,3) = 0.8963883995329452
Perror(2,3) = 0.5894680824318689
For s=0.1842105263157895:
Perror(1,2) = 0.7885705980050692
Perror(1,3) = 0.8018609615734384
Perror(2,3) = 0.5533894334999883
For s=0.22631578947368422:
Perror(1,2) = 0.7349121083463861
Perror(1,3) = 0.7219787094838493
Perror(2,3) = 0.524087004669471
For s=0.26842105263157895:
Perror(1,2) = 0.6894640539023564
Perror(1,3) = 0.6540818598261462
Perror(2,3) = 0.4997573549632219
For s=0.31052631578947365:
Perror(1,2) = 0.6510372019293418
Perror(1,3) = 0.5963025424296708
Perror(2,3) = 0.47925320697028106
For s=0.3526315789473684:
Perror(1,2) = 0.618819178917289
Perror(1,3) = 0.5472844423870448
Perror(2,3) = 0.4618046760909138
For s=0.39473684210526316:
Perror(1,2) = 0.5922335344766811
Perror(1,3) = 0.5060177996409827
Perror(2,3) = 0.44687504049641374
For s=0.436